<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_tower_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実行方法

1. 設定に絞り込みたいeNB-LCIDを入力
2. 上部メニューより「ランタイム」を選択
3. 「すべてのセルを実行」を選択
4. アップロードで一時停止しているので参照ボタンを押してTowerCollectorのCSVファイルをアップロード
5. 最後にマップが表示される

In [1]:
f = lambda enb, lcid: (enb << 8) | lcid

# 設定

In [2]:
# eNB-LCID

cells = [f(344574, 1)]

# 複数の場合
# cells = [f(737430, 10), f(737430, 11), f(737430, 12)]

cells

[88210945]

# アップロード

参照ボタンを押してCSVファイルをアップロード

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving 2022-05-05-23-40-47.csv to 2022-05-05-23-40-47.csv
User uploaded file "2022-05-05-23-40-47.csv" with length 52942 bytes


# プログラム

In [4]:
!pip install -U pandas

In [5]:
import pandas as pd
pd.options.plotting.backend = "plotly"

In [6]:
import folium
from folium.features import DivIcon
from folium import plugins

In [7]:
def make_map(df, zoom=14):

    lat = df["lat"].mean()
    lon = df["lon"].mean()

    map = folium.Map(tiles = None, location=[lat, lon], zoom_start=zoom)

    # Googleマップ標準
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map",
        attr="<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>",
    ).add_to(map)

    # Googleマップ航空写真
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map(航空写真)",
        attr="<a href='https://developers.google.com/maps/documentation'>© Google</a>",
    ).add_to(map)

    fg1 = folium.FeatureGroup(name="Good").add_to(map)
    fg2 = folium.FeatureGroup(name="Bad").add_to(map)

    for i, r in df.iterrows():

        # レイヤーしきい値
        condition = r["rsrp"] > -80

        folium.CircleMarker(
            location=[r.lat, r.lon],
            radius=10,
            color=r.color,
            fill=True,
            tooltip=r["id"],
        ).add_to(fg1 if condition else fg2)

        folium.Marker(
            location=[r.lat, r.lon],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
                html=f'<div style="text-align:center; font-size: 12pt; font-weight: bold">{r["ta"]}</div>',
            ),
            tooltip=f'{r["measured_at"]}<br />eNB-LCID:{r["id"]}<br />PCI:{r["psc"]}<br />TA:{r["ta"]}<br />RSRP:{r["rsrp"]}',
        ).add_to(fg1 if condition else fg2)
    
    folium.LayerControl().add_to(map)

    # DRAW
    folium.plugins.Draw(
        draw_options={"polygon": False, "rectangle": False, "circlemarker": False}
    ).add_to(map)

    return map

# データラングリング

In [8]:
df0 = pd.read_csv(fn, parse_dates=["measured_at"], dtype={"ta": "Int64"}).dropna(how="all", axis=1)
df0

,mcc,mnc,lac,cell_id,short_cell_id,rnc,psc,asu,dbm,ta,...,measured_at,net_type,neighboring,device,rsrp,rsrq,rssi,rssnr,ec_no,arfcn
0,440,11,13314,71694342,280056,6,260,17,-79,5,...,2022-05-05 06:57:51.906000+00:00,LTE,False,Xiaomi XIG02,-109.0,-11.0,-79.0,14.0,NaN,1500.0
1,440,11,13314,71694342,280056,6,260,15,-83,5,...,2022-05-05 06:58:27.885000+00:00,LTE,False,Xiaomi XIG02,-113.0,-11.0,-83.0,10.0,NaN,1500.0
2,440,11,13314,71694342,280056,6,260,18,-77,6,...,2022-05-05 06:59:03.863000+00:00,LTE,False,Xiaomi XIG02,-110.0,-12.0,-77.0,9.0,NaN,1500.0
3,440,11,13314,71694342,280056,6,260,16,-81,5,...,2022-05-05 07:01:14.854000+00:00,LTE,False,Xiaomi XIG02,-115.0,-13.0,-81.0,6.0,NaN,1500.0
4,440,11,13314,71694342,280056,6,260,18,-77,5,...,2022-05-05 07:01:32.863000+00:00,LTE,False,Xiaomi XIG02,-109.0,-11.0,-77.0,6.0,NaN,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:31.843000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
292,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:35.845000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
293,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:39.847000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
294,440,11,13315,71660291,279923,3,337,20,-73,6,...,2022-05-05 07:56:44.843000+00:00,LTE,False,Xiaomi XIG02,-106.0,-11.0,-73.0,6.0,NaN,1500.0


In [9]:
df0.dtypes

mcc                            int64
mnc                            int64
lac                            int64
cell_id                        int64
short_cell_id                  int64
rnc                            int64
psc                            int64
asu                            int64
dbm                            int64
ta                             Int64
lat                          float64
lon                          float64
accuracy                     float64
speed                        float64
bearing                      float64
altitude                     float64
measured_at      datetime64[ns, UTC]
net_type                      object
neighboring                     bool
device                        object
rsrp                         float64
rsrq                         float64
rssi                         float64
rssnr                        float64
ec_no                        float64
arfcn                        float64
dtype: object

In [10]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11").copy()
df1

,mcc,mnc,lac,cell_id,short_cell_id,rnc,psc,asu,dbm,ta,...,measured_at,net_type,neighboring,device,rsrp,rsrq,rssi,rssnr,ec_no,arfcn
0,440,11,13314,71694342,280056,6,260,17,-79,5,...,2022-05-05 06:57:51.906000+00:00,LTE,False,Xiaomi XIG02,-109.0,-11.0,-79.0,14.0,NaN,1500.0
1,440,11,13314,71694342,280056,6,260,15,-83,5,...,2022-05-05 06:58:27.885000+00:00,LTE,False,Xiaomi XIG02,-113.0,-11.0,-83.0,10.0,NaN,1500.0
2,440,11,13314,71694342,280056,6,260,18,-77,6,...,2022-05-05 06:59:03.863000+00:00,LTE,False,Xiaomi XIG02,-110.0,-12.0,-77.0,9.0,NaN,1500.0
3,440,11,13314,71694342,280056,6,260,16,-81,5,...,2022-05-05 07:01:14.854000+00:00,LTE,False,Xiaomi XIG02,-115.0,-13.0,-81.0,6.0,NaN,1500.0
4,440,11,13314,71694342,280056,6,260,18,-77,5,...,2022-05-05 07:01:32.863000+00:00,LTE,False,Xiaomi XIG02,-109.0,-11.0,-77.0,6.0,NaN,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:31.843000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
292,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:35.845000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
293,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:39.847000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
294,440,11,13315,71660291,279923,3,337,20,-73,6,...,2022-05-05 07:56:44.843000+00:00,LTE,False,Xiaomi XIG02,-106.0,-11.0,-73.0,6.0,NaN,1500.0


In [11]:
# 日時変換
df1["measured_at"] = df1["measured_at"].dt.tz_convert("Asia/Tokyo").dt.tz_localize(None)

In [12]:
df1["id"] = df1["short_cell_id"].astype(str) + "-" + df1["rnc"].astype(str)

In [13]:
# 色分け（青 > 緑 > 黄 > 橙 > 赤）

df1["color"] = pd.cut(
    df1["rsrp"],
    [-140, -100, -90, -80, -40],
    labels=["red", "orange", "yellow", "green"],
).astype(str)

idx = df1.groupby("cell_id")["rsrp"].transform(max) == df1["rsrp"]
df1.loc[idx, "color"] = "blue"

# 地図

## 全部

In [14]:
map1 = make_map(df1)
map1

In [15]:
map1.save("map1.html")

## 絞り込み

In [16]:
df2 = df1[df1["cell_id"].isin(cells)].copy()
df2[["measured_at", "id", "rsrp", "ta", "lat", "lon"]]

,measured_at,id,rsrp,ta,lat,lon
83,2022-05-05 16:12:11.643,344574-1,-103.0,41,35.766866,139.427825
84,2022-05-05 16:12:12.858,344574-1,-103.0,41,35.767126,139.428380
85,2022-05-05 16:12:17.855,344574-1,-103.0,41,35.767348,139.428981
86,2022-05-05 16:12:21.861,344574-1,-100.0,41,35.767321,139.429503
87,2022-05-05 16:12:25.860,344574-1,-100.0,41,35.767158,139.430032
88,2022-05-05 16:12:29.856,344574-1,-100.0,41,35.766981,139.430572
89,2022-05-05 16:12:33.931,344574-1,-116.0,42,35.766803,139.431115
90,2022-05-05 16:12:37.863,344574-1,-116.0,42,35.766667,139.431639
91,2022-05-05 16:12:41.865,344574-1,-110.0,42,35.766751,139.432146
92,2022-05-05 16:12:50.860,344574-1,-110.0,42,35.767512,139.432929


In [17]:
map2 = make_map(df2, 16)
map2

In [18]:
df2.plot(x="measured_at", y="rsrp")

In [19]:
df2[df2["ta"].notna()].plot(x="measured_at", y="ta")

In [20]:
map2.save("map2.html")

# ダウンロード

In [ ]:
from google.colab import files

files.download("map2.html")

# 時間帯指定

In [23]:
df3 = df2.set_index("measured_at").between_time("16:39:00", "16:41:00").reset_index()
df3

,measured_at,mcc,mnc,lac,cell_id,short_cell_id,rnc,psc,asu,dbm,...,neighboring,device,rsrp,rsrq,rssi,rssnr,ec_no,arfcn,id,color
0,2022-05-05 16:39:41.756,440,11,11265,88210945,344574,1,0,13,-87,...,False,Xiaomi XIG02,-120.0,-17.0,-87.0,-6.0,NaN,1575.0,344574-1,red
1,2022-05-05 16:39:41.855,440,11,11265,88210945,344574,1,0,13,-87,...,False,Xiaomi XIG02,-120.0,-17.0,-87.0,-6.0,NaN,1575.0,344574-1,red
2,2022-05-05 16:39:46.849,440,11,11265,88210945,344574,1,0,13,-87,...,False,Xiaomi XIG02,-120.0,-17.0,-87.0,-6.0,NaN,1575.0,344574-1,red
3,2022-05-05 16:39:52.847,440,11,11265,88210945,344574,1,0,12,-89,...,False,Xiaomi XIG02,-111.0,-10.0,-89.0,-4.0,NaN,1575.0,344574-1,red
4,2022-05-05 16:39:58.850,440,11,11265,88210945,344574,1,0,12,-89,...,False,Xiaomi XIG02,-111.0,-10.0,-89.0,-4.0,NaN,1575.0,344574-1,red
5,2022-05-05 16:40:03.848,440,11,11265,88210945,344574,1,0,15,-83,...,False,Xiaomi XIG02,-101.0,-3.0,-83.0,15.0,NaN,1575.0,344574-1,red
6,2022-05-05 16:40:08.847,440,11,11265,88210945,344574,1,0,15,-83,...,False,Xiaomi XIG02,-101.0,-3.0,-83.0,15.0,NaN,1575.0,344574-1,red
7,2022-05-05 16:40:13.845,440,11,11265,88210945,344574,1,0,17,-79,...,False,Xiaomi XIG02,-99.0,-3.0,-79.0,15.0,NaN,1575.0,344574-1,orange
8,2022-05-05 16:40:19.850,440,11,11265,88210945,344574,1,0,17,-79,...,False,Xiaomi XIG02,-99.0,-3.0,-79.0,15.0,NaN,1575.0,344574-1,orange


In [24]:
make_map(df3, 16)